# Corporacion Favorita Grocery Sales Forecasting
# General Prediction Pipeline
### Contains tuned variety of models used for the time series prediction:
#### Tree-based models (LightGBM, CatBoost, Random Forest, Extra Trees)
#### Linear model (Ridge Regression)
#### Recurrent Neural Network (see LSTM.ipynb for details)

In [1]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import Ridge
import tensorflow as tf
import lightgbm as lgb

# Load the data

In [2]:
%%time
df_train = pd.read_csv(
    'train_oiled.csv', 
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)
#df_train.drop('Unnamed: 0', axis = 1, inplace = True)
print('TRAIN LOADED')
df_test = pd.read_csv(
    "test_oiled.csv", 
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)
print('TEST LOADED')

items = pd.read_csv('data/items.csv')

df_train = df_train.merge(items, how='left', on='item_nbr', copy=False)
del items
gc.collect()

print('MERGED WITH ITEMS')

""""""
items = pd.read_csv(
    "data/items.csv",
).set_index("item_nbr")

TRAIN LOADED
TEST LOADED
MERGED WITH ITEMS
CPU times: user 3min 30s, sys: 12.2 s, total: 3min 42s
Wall time: 3min 42s


In [3]:
"""
    Unstack time series data to represent each time series as a row
    with store-item combination as an index;
"""

df_2017 = df_train.loc[df_train.date>=pd.datetime(2015,1,1)]
del df_train

promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train
gc.collect()

df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

In [4]:
"""
    Retrieve the data over the specified time period with a certain frequency
"""
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

"""
    Retrieve temporal features from preprocessed data ::
    
    Sales -      current value, mean values over n days, square mean values,
                 cumulative sales over n days, standard deviation in sales,
                 mean, std and squared mean for the last same n weekdays;
                 
    Promotions - same as for sales;
"""
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "SQUARE_mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values ** 2,
        "CUMULATIVE_3": get_timespan(df_2017, t2017, 3, 3).sum(axis=1).values,
        "std_3_2017": get_timespan(df_2017, t2017, 3, 3).std(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "CUMULATIVE_7": get_timespan(df_2017, t2017, 7, 7).sum(axis=1).values,
        "SQUARE_mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values ** 2,
        "std_7_2017": get_timespan(df_2017, t2017, 7, 7).std(axis=1).values,
        "SQUARE_std_7_2017": get_timespan(df_2017, t2017, 7, 7).std(axis=1).values ** 2,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "SQUARE_mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values ** 2,
        "std_14_2017": get_timespan(df_2017, t2017, 14, 14).std(axis=1).values,
        "CUMULATIVE_14": get_timespan(df_2017, t2017, 14, 14).sum(axis=1).values,
        "SQUARE_std_14_2017": get_timespan(df_2017, t2017, 14, 14).std(axis=1).values ** 2,
        
        "mean_21_2017": get_timespan(df_2017, t2017, 21, 21).mean(axis=1).values,
        "SQUARE_mean_21_2017": get_timespan(df_2017, t2017, 21, 21).mean(axis=1).values ** 2,
        "std_21_2017": get_timespan(df_2017, t2017, 21, 21).std(axis=1).values,
        "CUMULATIVE_21": get_timespan(df_2017, t2017, 21, 21).sum(axis=1).values,
        "SQUARE_std_21_2017": get_timespan(df_2017, t2017, 21, 21).std(axis=1).values ** 2,
        
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "SQUARE_mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values ** 2,
        "std_30_2017": get_timespan(df_2017, t2017, 30, 30).std(axis=1).values,
        "CUMULATIVE_30": get_timespan(df_2017, t2017, 30, 30).sum(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "CUMULATIVE_60": get_timespan(df_2017, t2017, 60, 60).sum(axis=1).values,
        "std_60_2017": get_timespan(df_2017, t2017, 60, 60).std(axis=1).values,
        "mean_90_2017": get_timespan(df_2017, t2017, 90, 90).mean(axis=1).values,
        "std_90_2017": get_timespan(df_2017, t2017, 90, 90).std(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "std_140_2017": get_timespan(df_2017, t2017, 140, 140).std(axis=1).values,
        
        ## PROMO _____________________________________________________________
        
        "promo_1_2017": get_timespan(promo_2017, t2017, 1, 1).values.ravel(),
        "promo_3_2017": get_timespan(promo_2017, t2017, 3, 3).sum(axis=1).values,
        "AVG_PROM_3": get_timespan(promo_2017, t2017, 3, 3).mean(axis=1).values,
        "proSTD_3_2017": get_timespan(promo_2017, t2017, 3, 3).std(axis=1).values,
        "promo_7_2017": get_timespan(promo_2017, t2017, 7, 7).sum(axis=1).values,
        "proSTD_7_2017": get_timespan(promo_2017, t2017, 7, 7).std(axis=1).values,
        "AVG_PROM_7": get_timespan(promo_2017, t2017, 7, 7).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "AVG_PROM_14": get_timespan(promo_2017, t2017, 14, 14).mean(axis=1).values,
        "proSTD_14_2017": get_timespan(promo_2017, t2017, 14, 14).std(axis=1).values,
        "promo_21_2017": get_timespan(promo_2017, t2017, 21, 21).sum(axis=1).values,
        #"AVG_PROM_21": get_timespan(promo_2017, t2017, 21, 21).mean(axis=1).values,
        #"proSTD_21_2017": get_timespan(promo_2017, t2017, 21, 21).std(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values
        #"AVG_PROM_60": get_timespan(promo_2017, t2017, 60, 60).mean(axis=1).values,
        #"promo_90_2017": get_timespan(promo_2017, t2017, 90, 90).sum(axis=1).values,
        #"promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values,
    })
    
    # weekday aggregations
    
    for i in range(7):
        X['prAWmo_4_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['SQUARE_prAWmo_4_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values ** 2
        #X['SQUARE_prAWmo_12_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values ** 2
        X['prAWmo_12_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        #X['prAWmo_20_dow{}_2017'.format(i)] = get_timespan(promo_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        X['mean_2_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 14-i, 2, freq='7D').mean(axis=1).values
        #X['SQUARE_mean_2_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 14-i, 2, freq='7D').mean(axis=1).values ** 2
        X['STD_2_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 14-i, 2, freq='7D').std(axis=1).values
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        #X['SQUARE_mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values ** 2
        X['STD_4_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').std(axis=1).values
        #X['mean_8_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 56-i, 8, freq='7D').mean(axis=1).values
        #X['STD_8_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 56-i, 8, freq='7D').std(axis=1).values
        #X['SQUARE_mean_8_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 56-i, 8, freq='7D').mean(axis=1).values ** 2
        X['mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values
        #X['STD_12_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').std(axis=1).values
        #X['SQUARE_mean_12_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 84-i, 12, freq='7D').mean(axis=1).values ** 2
        X['mean_16_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 112-i, 16, freq='7D').mean(axis=1).values
        X['STD_16_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 112-i, 16, freq='7D').std(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        #X['STD_20_DOW{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').std(axis=1).values
       
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

## Prepare the Train & Test sets

In [5]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
#t2017 = date(2016, 6, 5)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

X_test, y_test = prepare_dataset(date(2017, 7, 26))

# Validation 1 - weeks before the prediction period
X_val17, y_val17 = prepare_dataset(date(2017, 7, 26))
# Validation 2 - prediction period but 1 year ago
X_val16, y_val16 = prepare_dataset(date(2016, 8, 16)) 

X_test_sub = prepare_dataset(date(2017, 8, 16), is_train=False)
print(X_train.shape)

Preparing dataset...
(1032780, 113)


## Ridge Regression
#### The fastest and simplest model

In [ ]:
%%time
val17_pred = []
val16_pred = []
test_pred = []
cate_vars = []

for i in range(16):
    print("Step %d" % (i+1))
    #X_train, l
    label_train = y_train[:, i] # X_TR, Y_TR
    #X_val, 
    label_val16 = y_val16[:, i]
    
    bst = Ridge()
    
    bst.fit(X_train, label_train)
    val16_pred.append(bst.predict(X_val16))
    val17_pred.append(bst.predict(X_val17))
    test_pred.append(bst.predict(X_test))
    
nonneg = np.vectorize(lambda x: x if x >= 0 else 0)
val16_pred = nonneg(val16_pred)
val17_pred = nonneg(val17_pred)

y_cv = y_val16 * 0.8 + y_val17 * 0.2
pred_cv = val16_pred * 0.8 + val17_pred * 0.2

score_17 = mean_squared_log_error(y_val17, np.array(val17_pred).transpose()) ** 0.5
score_16 = mean_squared_log_error(y_val16, np.array(val16_pred).transpose()) ** 0.5
cv_score = score_16 * 0.8 + score_17 * 0.2

print("Val-2017:\t", 
    score_17)
print("1 year before:\t", 
    score_16)
print("CV Score:\t", 
    cv_score)

In [43]:
score_17 = mean_squared_log_error(y_val17, np.array(val17_pred).transpose()) ** 0.5
score_16 = mean_squared_log_error(y_val16, np.array(val16_pred).transpose()) ** 0.5
cv_score = score_16 * 0.8 + score_17 * 0.2

print("Val-2017:\t", 
    score_17)
print("1 year before:\t", 
    score_16)
print("CV Score:\t", 
    cv_score)

Val-2017:	 0.332954155156
1 year before:	 0.313901372647
CV Score:	 0.317711929149


In [9]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('RIDGE.csv', float_format='%.6f', index=None)

Making submission...


## Random Forest

In [10]:
from sklearn.ensemble import RandomForestRegressor as RFR

val_pred = []
test_pred = []
cate_vars = []

for i in range(16):
    print("Step %d" % (i+1))
    #X_train, l
    label_train = y_train[:, i] # X_TR, Y_TR
    #X_val, 
    label_val = y_val[:, i]

        
    bst = RFR(
            random_state = 1321,
            n_estimators = 40,
            max_features = .5,     
            max_depth = 11,  
            min_samples_split = 5,
            min_samples_leaf = 18,
            min_impurity_decrease = 0.0000001,
            n_jobs = 8
    )

    bst.fit(X_train, label_train)
    val_pred.append(bst.predict(X_val))
    test_pred.append(bst.predict(X_test))
print('')
print('$' * 50)
print("Validation mse:", 
    mean_squared_log_error(y_val, np.array(val_pred).transpose()) ** 0.5)

Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10
Step 11
Step 12
Step 13
Step 14
Step 15
Step 16

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Validation mse: 0.311087857164


In [11]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('FOREST_OIL.csv', float_format='%.6f', index=None)

Making submission...


## Extra Trees Regression
#### Similar to Random Forest, but in some cases works better when ensembled with other models

In [10]:
from sklearn.ensemble import ExtraTreesRegressor as ETR

val_pred = []
test_pred = []
cate_vars = []

# Train and predict for each of the 16 days required
for i in range(16):
    print("Step %d" % (i+1))
    #X_train, l
    label_train = y_train[:, i] 
    #X_val, 
    label_val = y_val[:, i]

        
    bst = ETR(
        random_state = 1321,
        n_estimators = 48,
        max_features = 0.650,      
        max_depth = 32,  
        min_samples_split = 2,
        min_samples_leaf = 9,
        min_impurity_decrease = 0.000001,
        n_jobs = 8,
        verbose = 1
    )

    bst.fit(X_train, label_train)
    val_pred.append(bst.predict(X_val))
    test_pred.append(bst.predict(X_test))
print('')
print('$' * 50)
print("Validation mse:", 
    mean_squared_log_error(y_val, np.array(val_pred).transpose()) ** 0.5)

Step 1


[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed: 19.2min finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    1.2s finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    1.1s finished


Step 2


[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed: 17.3min finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    1.2s finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    1.3s finished


Step 3


...

Step 15


[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed: 21.2min finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    1.6s finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    1.6s finished


Step 16


[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed: 21.3min finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    2.3s finished
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed:    2.4s finished



$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Validation mse: 0.310197330646


In [11]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('EXTRA.csv', float_format='%.6f', index=None)

Making submission...


## LightGBM
#### The best sole-performing model of the ensemble.
#### Feature relevance is tracked during LightGBM training.

In [11]:
%%time
print("Training and predicting models...")
params = {                             
    'num_leaves': 255,  
    'objective': 'regression_l2',   
    'max_depth': 15,                  
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.932,
    'bagging_fraction': 0.978,
    'bagging_freq': 5,
    'metric': 'rmse',
    'num_threads': 8
}
MAX_ROUNDS = 3500
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print('$' * 35)
    print("======== >> Step %d << ========" % (i+1))
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=10, verbose_eval=10
    )
    # Print model feature importances
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    del dtrain, dval, bst
    gc.collect()

print("Validation mse:", 
    mean_squared_log_error(y_val, np.array(val_pred).transpose()) ** 0.5)

Training and predicting models...
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
======== >> Step 1 << ========


/home/tameter/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 10 rounds.
[10]	training's rmse: 0.771835	valid_1's rmse: 0.753376
[20]	training's rmse: 0.633689	valid_1's rmse: 0.620259
[30]	training's rmse: 0.573778	valid_1's rmse: 0.56442
[40]	training's rmse: 0.548771	valid_1's rmse: 0.542664
[50]	training's rmse: 0.537821	valid_1's rmse: 0.534469
[60]	training's rmse: 0.532386	valid_1's rmse: 0.531306
[70]	training's rmse: 0.529084	valid_1's rmse: 0.530088
[80]	training's rmse: 0.52671	valid_1's rmse: 0.529589
[90]	training's rmse: 0.524753	valid_1's rmse: 0.52943
[100]	training's rmse: 0.522996	valid_1's rmse: 0.52937
[110]	training's rmse: 0.521406	valid_1's rmse: 0.529419
Early stopping, best iteration is:
[106]	training's rmse: 0.522044	valid_1's rmse: 0.529338
mean_14_2017: 2289867.22
CUMULATIVE_7: 1400501.10
AHEED3_1: 1391543.13
AHEED3_2: 1106061.92
SQUARE_mean_7_2017: 893846.24
CUMULATIVE_14: 364957.43
mean_7_2017: 346941.16
promo_0: 209510.65
AHEED0_2: 141858.51
mean_20_dow0_2017: 9790

[100]	training's rmse: 0.545474	valid_1's rmse: 0.5609
[110]	training's rmse: 0.543779	valid_1's rmse: 0.560586
[120]	training's rmse: 0.542215	valid_1's rmse: 0.560425
[130]	training's rmse: 0.540625	valid_1's rmse: 0.560302
[140]	training's rmse: 0.539255	valid_1's rmse: 0.560239
[150]	training's rmse: 0.537892	valid_1's rmse: 0.560126
[160]	training's rmse: 0.53667	valid_1's rmse: 0.559975
[170]	training's rmse: 0.535491	valid_1's rmse: 0.559955
[180]	training's rmse: 0.534277	valid_1's rmse: 0.559972
Early stopping, best iteration is:
[170]	training's rmse: 0.535491	valid_1's rmse: 0.559955
SQUARE_mean_14_2017: 2212575.21
mean_14_2017: 1486237.05
AHEED3_2: 890704.52
CUMULATIVE_14: 564161.69
CUMULATIVE_7: 397560.79
SQUARE_mean_7_2017: 317003.59
AHEED0_1: 268979.80
AHEED7_6: 186203.71
promo_1: 139873.17
mean_30_2017: 103669.63
AHEED7_1: 101774.23
CUMULATIVE_30: 85892.83
mean_20_dow1_2017: 81169.35
AHEED3_1: 76340.96
SQUARE_mean_30_2017: 66874.38
mean_8_dow1_2017: 66754.87
mean_60_201

[120]	training's rmse: 0.542064	valid_1's rmse: 0.572786
[130]	training's rmse: 0.540541	valid_1's rmse: 0.57277
[140]	training's rmse: 0.539118	valid_1's rmse: 0.572602
[150]	training's rmse: 0.537799	valid_1's rmse: 0.572553
[160]	training's rmse: 0.536479	valid_1's rmse: 0.572459
Early stopping, best iteration is:
[159]	training's rmse: 0.536604	valid_1's rmse: 0.572454
CUMULATIVE_14: 2540517.99
mean_14_2017: 1844839.84
mean_8_dow2_2017: 668224.03
AHEED7_1: 637637.10
SQUARE_mean_14_2017: 612104.65
SQUARE_mean_8_dow2_2017: 494768.13
CUMULATIVE_7: 389506.41
mean_7_2017: 298510.07
promo_2: 224629.68
mean_20_dow2_2017: 223277.35
CUMULATIVE_30: 114781.26
AHEED0_1: 77797.61
SQUARE_mean_30_2017: 66012.56
AHEED0_2: 57204.81
mean_12_dow2_2017: 54654.77
SQUARE_mean_12_dow2_2017: 53578.53
SQUARE_std_14_2017: 52166.13
mean_30_2017: 44655.58
mean_16_dow2_2017: 41745.65
prAWmo_20_dow2_2017: 39151.90
SQUARE_mean_7_2017: 26318.77
AHEED3_2: 25636.37
AVG_PROM_14: 22884.66
prAWmo_4_dow2_2017: 18409.82

...

CUMULATIVE_21: 3426871.31
SQUARE_mean_21_2017: 2104424.41
mean_8_dow3_2017: 986085.45
CUMULATIVE_30: 645568.16
SQUARE_mean_8_dow3_2017: 509409.42
SQUARE_mean_30_2017: 286373.15
promo_10: 232756.67
mean_20_dow3_2017: 200982.11
mean_30_2017: 171779.75
AHEED7_1: 171598.36
AHEED0_1: 134571.60
AHEED7_2: 103408.78
SQUARE_mean_12_dow3_2017: 81160.95
mean_12_dow3_2017: 79317.85
SQUARE_std_21_2017: 65978.32
AHEED0_2: 48402.10
mean_60_2017: 47486.14
SQUARE_mean_7_2017: 38207.78
CUMULATIVE_14: 33564.63
mean_4_dow3_2017: 31973.04
AHEED3_4: 31028.22
CUMULATIVE_60: 28963.46
mean_7_2017: 23939.10
mean_21_2017: 20769.46
CUMULATIVE_7: 20274.52
AVG_PROM_21: 20215.23
AHEED0_3: 15220.50
std_30_2017: 14603.74
???BF_unpromo_7: 13767.06
STD_NAFTA_30_2017: 13668.74
???BF_unpromo_0: 12345.01
???BF_unpromo_10: 12106.90
promo_12: 11877.69
mean_140_2017: 11860.04
SQUARE_mean_4_dow3_2017: 11149.94
AHEED7_3: 11061.45
mean_16_dow3_2017: 10667.46
promo_14: 10533.31
mean_8_dow4_2017: 10059.32
CUM_NAFTA_30_2017: 9699.6

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
======== >> Step 13 << ========
Training until validation scores don't improve for 10 rounds.
[10]	training's rmse: 0.800694	valid_1's rmse: 0.803042
[20]	training's rmse: 0.680092	valid_1's rmse: 0.686013
[30]	training's rmse: 0.628472	valid_1's rmse: 0.636746
[40]	training's rmse: 0.606556	valid_1's rmse: 0.616861
[50]	training's rmse: 0.596328	valid_1's rmse: 0.608752
[60]	training's rmse: 0.590803	valid_1's rmse: 0.605387
[70]	training's rmse: 0.587128	valid_1's rmse: 0.603793
[80]	training's rmse: 0.584381	valid_1's rmse: 0.602945
[90]	training's rmse: 0.581999	valid_1's rmse: 0.602488
[100]	training's rmse: 0.579896	valid_1's rmse: 0.602257
[110]	training's rmse: 0.577988	valid_1's rmse: 0.602006
[120]	training's rmse: 0.576189	valid_1's rmse: 0.601908
[130]	training's rmse: 0.574462	valid_1's rmse: 0.601843
Early stopping, best iteration is:
[128]	training's rmse: 0.574777	valid_1's rmse: 0.601827
CUMULATIVE_30: 2149212.02
SQUARE_mean_21_2017:

[50]	training's rmse: 0.586532	valid_1's rmse: 0.59553
[60]	training's rmse: 0.581359	valid_1's rmse: 0.592482
[70]	training's rmse: 0.577955	valid_1's rmse: 0.591232
[80]	training's rmse: 0.575355	valid_1's rmse: 0.590628
[90]	training's rmse: 0.573163	valid_1's rmse: 0.590262
[100]	training's rmse: 0.571196	valid_1's rmse: 0.590086
[110]	training's rmse: 0.569416	valid_1's rmse: 0.5901
Early stopping, best iteration is:
[105]	training's rmse: 0.570264	valid_1's rmse: 0.590043
CUMULATIVE_30: 1737390.56
CUMULATIVE_21: 1213786.91
mean_8_dow6_2017: 870543.55
SQUARE_mean_21_2017: 798773.94
SQUARE_mean_30_2017: 786377.18
mean_30_2017: 675335.41
promo_13: 354305.75
AHEED3_1: 255452.88
SQUARE_mean_8_dow6_2017: 242718.26
mean_20_dow6_2017: 173357.09
AHEED3_2: 126486.95
mean_12_dow6_2017: 102894.95
mean_21_2017: 84483.94
mean_60_2017: 37316.71
CUMULATIVE_60: 36208.48
AHEED0_1: 25821.03
prAWmo_20_dow6_2017: 23284.76
mean_140_2017: 19971.80
promo_10: 17059.86
AHEED7_4: 17025.89
prAWmo_4_dow6_201

CUMULATIVE_21: 2092858.71
mean_30_2017: 1624509.00
SQUARE_mean_21_2017: 775849.98
SQUARE_mean_8_dow0_2017: 684907.53
mean_8_dow0_2017: 470823.98
promo_14: 470509.12
SQUARE_mean_30_2017: 433752.97
CUMULATIVE_30: 310094.54
mean_21_2017: 280993.99
AHEED3_1: 236830.41
mean_20_dow0_2017: 174272.52
mean_14_2017: 121552.68
prAWmo_20_dow0_2017: 68578.47
CUMULATIVE_7: 62599.20
CUMULATIVE_60: 58798.47
mean_60_2017: 53788.98
SQUARE_mean_12_dow0_2017: 46051.42
prAWmo_4_dow0_2017: 35890.71
mean_7_2017: 21462.62
SQUARE_mean_7_2017: 20798.74
mean_12_dow0_2017: 20248.02
std_90_2017: 20241.15
std_140_2017: 19567.92
mean_140_2017: 18362.68
mean_16_dow0_2017: 17539.68
SQUARE_prAWmo_12_dow0_2017: 16295.11
AHEED0_5: 15963.57
SQUARE_mean_14_2017: 12695.62
promo_10: 12572.43
AHEED3_2: 11588.36
CUM_NAFTA_90_2017: 10385.93
std_30_2017: 10322.75
AHEED0_2: 10211.00
STD_20_DOW0_2017: 9727.53
mean_4_dow0_2017: 9396.73
AHEED0_1: 9049.02
SQUARE_std_21_2017: 8921.90
STD_16_DOW0_2017: 8731.64
promo_7: 8218.19
AVG_PROM

Validation mse: 0.309044635115
CPU times: user 9h 30min 39s, sys: 3min 46s, total: 9h 34min 26s
Wall time: 1h 38min 51s


# CatBoost
#### A new Gradient Boosting implementation by Yandex. 

In [20]:
val_pred = []
test_pred = []
cate_vars = []

for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    model = CatBoostRegressor(
        iterations=131, 
        learning_rate=0.18,
        thread_count = 8,
        loss_function = 'RMSE',
        eval_metric = 'RMSE',
        use_best_model = True,
        random_seed = 1321,
        l2_leaf_reg = 0.2,
        depth=10)

    model.fit(
        X_train, y_train[:, i],
        cat_features=cate_vars, 
        verbose = 2,
        eval_set=(X_val, y_val[:, i])
    )

    val_pred.append(model.predict(X_val))
    test_pred.append(model.predict(X_test))
nonneg = np.vectorize(lambda x: x if x >= 0 else 0)
val_pred = nonneg(val_pred)
test_pred = nonneg(test_pred)

print("VAL_RMSLE:", 
    mean_squared_log_error(y_val, np.array(val_pred).transpose()) ** 0.5)

Step 1


/home/tameter/anaconda3/lib/python3.6/site-packages/catboost/core.py:533: FutureWarning: The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').
  warnings.warn("The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').", FutureWarning)


0:	learn: 1.2336633	test: 1.1878087	best: 1.1878087 (0)	total: 5.3s	remaining: 11m 29s
1:	learn: 1.0653618	test: 1.0233769	best: 1.0233769 (1)	total: 10.2s	remaining: 10m 56s
2:	learn: 0.9335813	test: 0.8965686	best: 0.8965686 (2)	total: 15.3s	remaining: 10m 54s
3:	learn: 0.8324293	test: 0.7987287	best: 0.7987287 (3)	total: 20.6s	remaining: 10m 55s
4:	learn: 0.7554325	test: 0.7245090	best: 0.7245090 (4)	total: 26s	remaining: 10m 54s
5:	learn: 0.6983373	test: 0.6702406	best: 0.6702406 (5)	total: 31.2s	remaining: 10m 49s
6:	learn: 0.6556100	test: 0.6302213	best: 0.6302213 (6)	total: 36.5s	remaining: 10m 45s
7:	learn: 0.6252508	test: 0.6022709	best: 0.6022709 (7)	total: 41.9s	remaining: 10m 43s
8:	learn: 0.6026015	test: 0.5818401	best: 0.5818401 (8)	total: 47.1s	remaining: 10m 39s
9:	learn: 0.5868985	test: 0.5680868	best: 0.5680868 (9)	total: 52.4s	remaining: 10m 33s
10:	learn: 0.5759551	test: 0.5587996	best: 0.5587996 (10)	total: 57.7s	remaining: 10m 29s
11:	learn: 0.5675472	test: 0.5516

92:	learn: 0.5358420	test: 0.5319379	best: 0.5319379 (92)	total: 9m 26s	remaining: 3m 51s
93:	learn: 0.5357700	test: 0.5319610	best: 0.5319379 (92)	total: 9m 33s	remaining: 3m 45s
94:	learn: 0.5356959	test: 0.5319512	best: 0.5319379 (92)	total: 9m 41s	remaining: 3m 40s
95:	learn: 0.5355576	test: 0.5319058	best: 0.5319058 (95)	total: 9m 49s	remaining: 3m 34s
96:	learn: 0.5354029	test: 0.5319048	best: 0.5319048 (96)	total: 9m 55s	remaining: 3m 28s
97:	learn: 0.5353388	test: 0.5319096	best: 0.5319048 (96)	total: 10m 2s	remaining: 3m 22s
98:	learn: 0.5352069	test: 0.5318370	best: 0.5318370 (98)	total: 10m 8s	remaining: 3m 16s
99:	learn: 0.5350428	test: 0.5318642	best: 0.5318370 (98)	total: 10m 13s	remaining: 3m 10s
100:	learn: 0.5349172	test: 0.5317916	best: 0.5317916 (100)	total: 10m 19s	remaining: 3m 4s
101:	learn: 0.5347814	test: 0.5317765	best: 0.5317765 (101)	total: 10m 25s	remaining: 2m 57s
102:	learn: 0.5347000	test: 0.5317510	best: 0.5317510 (102)	total: 10m 31s	remaining: 2m 51s
1

50:	learn: 0.5606756	test: 0.5628861	best: 0.5628861 (50)	total: 4m 40s	remaining: 7m 20s
51:	learn: 0.5605416	test: 0.5628085	best: 0.5628085 (51)	total: 4m 45s	remaining: 7m 14s
52:	learn: 0.5604633	test: 0.5627989	best: 0.5627989 (52)	total: 4m 51s	remaining: 7m 8s
53:	learn: 0.5602766	test: 0.5627138	best: 0.5627138 (53)	total: 4m 56s	remaining: 7m 3s
54:	learn: 0.5601791	test: 0.5627011	best: 0.5627011 (54)	total: 5m 2s	remaining: 6m 57s
55:	learn: 0.5600946	test: 0.5626542	best: 0.5626542 (55)	total: 5m 7s	remaining: 6m 51s
56:	learn: 0.5599374	test: 0.5625222	best: 0.5625222 (56)	total: 5m 12s	remaining: 6m 45s
57:	learn: 0.5598090	test: 0.5624937	best: 0.5624937 (57)	total: 5m 17s	remaining: 6m 40s
58:	learn: 0.5597370	test: 0.5624531	best: 0.5624531 (58)	total: 5m 22s	remaining: 6m 34s
59:	learn: 0.5596250	test: 0.5623306	best: 0.5623306 (59)	total: 5m 28s	remaining: 6m 28s
60:	learn: 0.5595430	test: 0.5623005	best: 0.5623005 (60)	total: 5m 33s	remaining: 6m 23s
61:	learn: 0.5

...

6:	learn: 0.6910622	test: 0.6902231	best: 0.6902231 (6)	total: 37.1s	remaining: 10m 57s
7:	learn: 0.6646066	test: 0.6648704	best: 0.6648704 (7)	total: 42.4s	remaining: 10m 52s
8:	learn: 0.6452239	test: 0.6460162	best: 0.6460162 (8)	total: 47.4s	remaining: 10m 42s
9:	learn: 0.6316295	test: 0.6330331	best: 0.6330331 (9)	total: 52.8s	remaining: 10m 38s
10:	learn: 0.6221553	test: 0.6238157	best: 0.6238157 (10)	total: 58.1s	remaining: 10m 33s
11:	learn: 0.6150360	test: 0.6168872	best: 0.6168872 (11)	total: 1m 3s	remaining: 10m 29s
12:	learn: 0.6095635	test: 0.6116238	best: 0.6116238 (12)	total: 1m 8s	remaining: 10m 24s
13:	learn: 0.6056918	test: 0.6079347	best: 0.6079347 (13)	total: 1m 14s	remaining: 10m 19s
14:	learn: 0.6029697	test: 0.6052788	best: 0.6052788 (14)	total: 1m 19s	remaining: 10m 11s
15:	learn: 0.6009481	test: 0.6032632	best: 0.6032632 (15)	total: 1m 24s	remaining: 10m 5s
16:	learn: 0.5993788	test: 0.6017064	best: 0.6017064 (16)	total: 1m 29s	remaining: 9m 58s
17:	learn: 0.598

98:	learn: 0.5823963	test: 0.5927078	best: 0.5926275 (94)	total: 9m 12s	remaining: 2m 58s
99:	learn: 0.5823135	test: 0.5927138	best: 0.5926275 (94)	total: 9m 17s	remaining: 2m 52s
100:	learn: 0.5822355	test: 0.5927099	best: 0.5926275 (94)	total: 9m 22s	remaining: 2m 47s
101:	learn: 0.5821443	test: 0.5926900	best: 0.5926275 (94)	total: 9m 28s	remaining: 2m 41s
102:	learn: 0.5820084	test: 0.5926956	best: 0.5926275 (94)	total: 9m 34s	remaining: 2m 36s
103:	learn: 0.5819154	test: 0.5926764	best: 0.5926275 (94)	total: 9m 39s	remaining: 2m 30s
104:	learn: 0.5818584	test: 0.5926660	best: 0.5926275 (94)	total: 9m 45s	remaining: 2m 25s
105:	learn: 0.5816731	test: 0.5926564	best: 0.5926275 (94)	total: 9m 51s	remaining: 2m 19s
106:	learn: 0.5815865	test: 0.5926647	best: 0.5926275 (94)	total: 9m 57s	remaining: 2m 13s
107:	learn: 0.5814607	test: 0.5926707	best: 0.5926275 (94)	total: 10m 2s	remaining: 2m 8s
108:	learn: 0.5813088	test: 0.5926484	best: 0.5926275 (94)	total: 10m 8s	remaining: 2m 2s
109

56:	learn: 0.5774566	test: 0.5818070	best: 0.5818070 (56)	total: 6m 39s	remaining: 8m 38s
57:	learn: 0.5772919	test: 0.5817589	best: 0.5817589 (57)	total: 6m 46s	remaining: 8m 31s
58:	learn: 0.5772008	test: 0.5817383	best: 0.5817383 (58)	total: 6m 52s	remaining: 8m 23s
59:	learn: 0.5770989	test: 0.5816967	best: 0.5816967 (59)	total: 6m 59s	remaining: 8m 16s
60:	learn: 0.5769720	test: 0.5816414	best: 0.5816414 (60)	total: 7m 7s	remaining: 8m 10s
61:	learn: 0.5768754	test: 0.5815975	best: 0.5815975 (61)	total: 7m 15s	remaining: 8m 4s
62:	learn: 0.5767881	test: 0.5815788	best: 0.5815788 (62)	total: 7m 21s	remaining: 7m 56s
63:	learn: 0.5766561	test: 0.5815241	best: 0.5815241 (63)	total: 7m 27s	remaining: 7m 48s
64:	learn: 0.5764436	test: 0.5814677	best: 0.5814677 (64)	total: 7m 32s	remaining: 7m 39s
65:	learn: 0.5763431	test: 0.5814412	best: 0.5814412 (65)	total: 7m 38s	remaining: 7m 31s
66:	learn: 0.5761994	test: 0.5814450	best: 0.5814412 (65)	total: 7m 45s	remaining: 7m 24s
67:	learn: 0

14:	learn: 0.6007983	test: 0.6131244	best: 0.6131244 (14)	total: 1m 27s	remaining: 11m 16s
15:	learn: 0.5987955	test: 0.6110723	best: 0.6110723 (15)	total: 1m 33s	remaining: 11m 11s
16:	learn: 0.5976509	test: 0.6097154	best: 0.6097154 (16)	total: 1m 38s	remaining: 11m 2s
17:	learn: 0.5967282	test: 0.6086278	best: 0.6086278 (17)	total: 1m 44s	remaining: 10m 56s
18:	learn: 0.5958727	test: 0.6077639	best: 0.6077639 (18)	total: 1m 50s	remaining: 10m 48s
19:	learn: 0.5950692	test: 0.6068718	best: 0.6068718 (19)	total: 1m 55s	remaining: 10m 43s
20:	learn: 0.5945924	test: 0.6063795	best: 0.6063795 (20)	total: 2m 1s	remaining: 10m 39s
21:	learn: 0.5941151	test: 0.6058545	best: 0.6058545 (21)	total: 2m 7s	remaining: 10m 30s
22:	learn: 0.5936788	test: 0.6054735	best: 0.6054735 (22)	total: 2m 13s	remaining: 10m 25s
23:	learn: 0.5932864	test: 0.6050827	best: 0.6050827 (23)	total: 2m 19s	remaining: 10m 20s
24:	learn: 0.5929348	test: 0.6048123	best: 0.6048123 (24)	total: 2m 25s	remaining: 10m 15s
25

106:	learn: 0.5811883	test: 0.6000821	best: 0.6000821 (106)	total: 10m 28s	remaining: 2m 20s
107:	learn: 0.5811244	test: 0.6000746	best: 0.6000746 (107)	total: 10m 33s	remaining: 2m 15s
108:	learn: 0.5810262	test: 0.6000708	best: 0.6000708 (108)	total: 10m 39s	remaining: 2m 9s
109:	learn: 0.5809279	test: 0.6000722	best: 0.6000708 (108)	total: 10m 45s	remaining: 2m 3s
110:	learn: 0.5808509	test: 0.6000440	best: 0.6000440 (110)	total: 10m 51s	remaining: 1m 57s
111:	learn: 0.5807289	test: 0.6000311	best: 0.6000311 (111)	total: 10m 57s	remaining: 1m 51s
112:	learn: 0.5806403	test: 0.6000073	best: 0.6000073 (112)	total: 11m 3s	remaining: 1m 45s
113:	learn: 0.5804956	test: 0.5999704	best: 0.5999704 (113)	total: 11m 9s	remaining: 1m 39s
114:	learn: 0.5803709	test: 0.6000017	best: 0.5999704 (113)	total: 11m 15s	remaining: 1m 33s
115:	learn: 0.5802958	test: 0.5999524	best: 0.5999524 (115)	total: 11m 20s	remaining: 1m 27s
116:	learn: 0.5802495	test: 0.5999495	best: 0.5999495 (116)	total: 11m 26s

In [21]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('CAT_OIL.csv', float_format='%.6f', index=None)

Making submission...
